In [14]:
# Basic Packages
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

from simple_colors import *

# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [11]:
# Getting the Data
os.getcwd()
db  = pd.read_csv("Data/stud.csv")

In [12]:
# Reading Data
db.head(2)

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88


In [15]:
"""
Displaying unique contents in a columns
"""


# Gender
print(green("Unique Gender Values :",'bold'),db['gender'].unique())

# Etnicity
print(green("\nUnique race_ethnicity Values :",'bold'),db['race_ethnicity'].unique())

# Education
print(green("\nUnique Education Values :",'bold'),db['parental_level_of_education'].unique())

# Lunch
print(green("\nUnique Lunch Values :",'bold'),db['lunch'].unique())

# Preparation
print(green("\nUnique Preparation Values :",'bold'),db['test_preparation_course'].unique())

Unique Gender Values : ['female' 'male']

Unique race_ethnicity Values : ['group B' 'group C' 'group A' 'group D' 'group E']

Unique Education Values : ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']

Unique Lunch Values : ['standard' 'free/reduced']

Unique Preparation Values : ['none' 'completed']


##### Preparing X and Y Variable

In [37]:
# X & Y Variables
X= df.drop(columns=['math_score'],axis=1)
y = df.math_score

In [42]:
# Checkin Variables are Created properly
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [44]:
# Create Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns



In [45]:
"""
OneHotEncoding for Categorical Values.
Scaling for numerical values which STD. DEV (1) Mean is 0
ColumnTransform for mixed values both numerical and categorical Values

"""
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_tranformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
    ("OneHotEncoder",oh_transformer,cat_features),
    ("StandardScalar",numeric_tranformer,num_features)
     ]
)


In [46]:
X = preprocessor.fit_transform(X)

In [52]:
X.shape

(1000, 19)

In [55]:
# Seperating Dat into test train Data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=15)


# Sizes of train test Data
X_test.shape,X_train.shape

((200, 19), (800, 19))

##### Evaluating Models and Metrics 

In [80]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [81]:
# Testing all models to determine which is the best suitable.
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')





Linear Regression
Model performance for Training set
- Root Mean Squared Error: 5.2905
- Mean Absolute Error: 4.2140
- R2 Score: 0.8788
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.4754
- Mean Absolute Error: 4.4052
- R2 Score: 0.8655


Lasso
Model performance for Training set
- Root Mean Squared Error: 6.5076
- Mean Absolute Error: 5.1400
- R2 Score: 0.8166
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 6.7212
- Mean Absolute Error: 5.2705
- R2 Score: 0.7974


Ridge
Model performance for Training set
- Root Mean Squared Error: 5.2890
- Mean Absolute Error: 4.2120
- R2 Score: 0.8789
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.4808
- Mean Absolute Error: 4.4050
- R2 Score: 0.8652


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 5.8270
- Mean Absolute Error: 4.5705
- R2 Score: 0.8530
-----------------------